<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Converting ADM Initial Data in the Spherical or Cartesian Basis to BSSN Initial Data in the Desired Curvilinear Basis
## Author: Zach Etienne

[comment]: <> (Abstract: TODO)

### This module is meant for use with any initial data that can be represented numerically in ADM form, either in the Spherical or Cartesian basis. I.e., the ADM variables are given $\left\{\gamma_{ij}, K_{ij}, \alpha, \beta^i\right\}$ *numerically* as functions of $(r,\theta,\phi)$ or $(x,y,z)$; e.g., through an initial data solver.

**Notebook Status:** <font color='orange'><b> Self-Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). **Additional validation tests may have been performed, but are as yet, undocumented. (TODO)**

### NRPy+ Source Code for this module: [BSSN/Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.py](../edit/BSSN/Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.py)



## Introduction:
Given the ADM variables:

$$\left\{\gamma_{ij}, K_{ij}, \alpha, \beta^i, B^i\right\}$$

in the Spherical or Cartesian basis, and as functions of $(r,\theta,\phi)$ or $(x,y,z)$, respectively, this module documents their conversion to the BSSN variables

$$\left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\},$$ 

in the desired curvilinear basis (given by `reference_metric::CoordSystem`). Then it rescales the resulting BSSNCurvilinear variables (as defined in [the BSSN Curvilinear tutorial](Tutorial-BSSNCurvilinear.ipynb)) into the form needed for BSSNCurvilinear evolutions:

$$\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\}.$$ 

We will use as our core example in this module UIUC initial data, which are ([as documented in their NRPy+ initial data module](Tutorial-ADM_Initial_Data-UIUC_BlackHole.ipynb)) given in terms of ADM variables in Spherical coordinates.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2](#read_convert_adm_to_cartesian_bssn): `initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart()`: Read or compute ADM variables at all points on all grids, and convert them to BSSN curvilinear
1. [Step 3](#basis_xform): `initial_data_BSSN_basis_transform_Cartesian_to_rfm()`: Convert BSSN vectors/tensors from Cartesian to reference-metric basis
1. [Step 4](#lambda): `initial_data_lambdaU_grid_interior()`: Compute $\lambda^i$ from finite-difference derivatives of rescaled metric quantities
1. [Step 5](#nbd): `register_NRPy_basic_defines()`: Register `ID_data_struct` with `NRPy_basic_defines.h
1. [Step 6](#code_validation):Code Validation against  `BSSN.ADM_Initial_Data_Reader__BSSN_Converter` NRPy+ module
1. [Step 7](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
# Step 1: Initialize core Python/NRPy+ modules
from outputC import outputC,lhrh,add_to_Cfunction_dict # NRPy+: Core C code output module
from outputC import outputC_register_C_functions_and_NRPy_basic_defines, outC_NRPy_basic_defines_h_dict
from outputC import NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines
from outputC import construct_NRPy_function_prototypes_h, construct_NRPy_basic_defines_h
import NRPy_param_funcs as par    # NRPy+: Parameter interface
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends
import finite_difference as fin   # NRPy+: Finite difference C code generation module
import grid as gri                # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp          # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm    # NRPy+: Reference metric support
import BSSN.BSSN_quantities as Bq # NRPy+: Computes useful BSSN quantities; e.g., gammabarUU & GammabarUDD needed below
import cmdline_helper as cmd      # NRPy+: Multi-platform Python command-line interface
from pickling import pickle_NRPy_env # NRPy+: Pickle/unpickle NRPy+ environment, for parallel codegen
import os, shutil, sys            # Standard Python modules for multiplatform OS-level functions

# Step 1.a: Create output directory for C codes generated by this tutorial:
Ccodesrootdir = os.path.join("ADM_initial_data_to_BSSN_converter")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
# !rm -r ScalarWaveCurvilinear_Playground_Ccodes
shutil.rmtree(Ccodesrootdir, ignore_errors=True)
# Then create a fresh directory
cmd.mkdir(Ccodesrootdir)

<a id='read_convert_adm_to_cartesian_bssn'></a>

# Step 2: `initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart()`: Read or compute ADM variables at all points on all grids, and convert them to BSSN curvilinear \[Back to [top](#toc)\]
$$\label{read_convert_adm_to_cartesian_bssn}$$

`initial_data_reader__convert_to_BSSN_from_ADM_Spherical()` and `initial_data_reader__convert_to_BSSN_from_ADM_Cartesian()` are the primary initial data read-in routines for NRPy+. These routines do the following:

1. Call `ID_function()` to read or compute ADM initial data at Cartesian point `xCart[3]`=$(x,y,z)$, in the spherical or Cartesian basis (`initial_data_reader__convert_to_BSSN_from_ADM_Spherical()` or `initial_data_reader__convert_to_BSSN_from_ADM_Cartesian()` respectively.
1. Convert ADM variables from the spherical or Cartesian basis to the Cartesian basis.
1. Convert ADM variables in the Cartesian basis to BSSN variables in the Cartesian basis.
1. Call `initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale()` to convert all BSSN vectors/tensors (*except* $\lambda^i$) in the Cartesian basis, to the basis specified by `reference_metric::CoordSystem`.
1. Call `initial_data_lambdaU_grid_interior()` to compute $\lambda^i$ in the `reference_metric::CoordSystem` basis, *in the grid interior only*.

**Important Note**: After `initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart()` is called, inner/outer boundary conditions must be applied to $\lambda^i$ to ensure it is specified on the grid boundaries.

In [2]:
def add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart(input_Coord="Spherical"):
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"

    output_Coord = par.parval_from_str("reference_metric::CoordSystem")
    desc = "Read in ADM initial data in the " + input_Coord + " basis, and convert to BSSN data in " + output_Coord + " coordinates"
    name = "initial_data_reader__convert_to_BSSN_from_ADM_" + input_Coord
    params = """griddata_struct *restrict griddata, ID_persist_struct *restrict ID_persist,
                                                             void ID_function(const paramstruct *params, const REAL xCart[3],
                                                                              const ID_persist_struct *restrict ID_persist,
                                                                              ID_output_struct *restrict ID_output)"""

    body = r"""
  const int Nxx_plus_2NGHOSTS0 = griddata->params.Nxx_plus_2NGHOSTS0;
  const int Nxx_plus_2NGHOSTS1 = griddata->params.Nxx_plus_2NGHOSTS1;
  const int Nxx_plus_2NGHOSTS2 = griddata->params.Nxx_plus_2NGHOSTS2;

  LOOP_OMP("omp parallel for", i0,0,Nxx_plus_2NGHOSTS0, i1,0,Nxx_plus_2NGHOSTS1, i2,0,Nxx_plus_2NGHOSTS2) {
    // xCart is the global Cartesian coordinate, which accounts for any grid offsets from the origin.
    REAL xCart[3];  xx_to_Cart(&griddata->params, griddata->xx, i0,i1,i2, xCart);

    // Read or compute initial data at destination point xCart
    ID_output_struct ID_output;
    ID_function(&griddata->params, xCart, ID_persist, &ID_output);

    // Unpack ID_output for scalar alpha
    const REAL alpha = ID_output.alpha;

    // Unpack ID_output for ADM vectors/tensors
"""
    for i in ["betaSphorCartU", "BSphorCartU"]:
        for j in range(3):
            varname = i + str(j)
            body += "    const REAL " + varname + " = ID_output." + varname + ";\n"
        body += "\n"
    for i in ["gammaSphorCartDD", "KSphorCartDD"]:
        for j in range(3):
            for k in range(j, 3):
                varname = i + str(j) + str(k)
                body += "    const REAL " + varname + " = ID_output." + varname + ";\n"
        body += "\n"

    body += r"""
    // Perform the basis transform on ADM vectors/tensors from """+input_Coord+""" to Cartesian:
    //   (Don't be surprised if it's trivial when ADM quantities are already in the Cartesian basis.)
    REAL betaCartU0,betaCartU1,betaCartU2;
    REAL BCartU0,BCartU1,BCartU2;
    REAL gammaCartDD00,gammaCartDD01,gammaCartDD02,gammaCartDD11,gammaCartDD12,gammaCartDD22;
    REAL KCartDD00,KCartDD01,KCartDD02,KCartDD11,KCartDD12,KCartDD22;
    {
      // Set destination point
      const REAL Cartx = xCart[0];
      const REAL Carty = xCart[1];
      const REAL Cartz = xCart[2];

      // Set destination xx[3]
"""
    # Set reference_metric to the input_Coord
    par.set_parval_from_str("reference_metric::CoordSystem", input_Coord)
    rfm.reference_metric()

    body += outputC(rfm.Cart_to_xx[:3], ["const REAL xx0", "const REAL xx1", "const REAL xx2"],
                    filename="returnstring",
                    params="outCverbose=False,includebraces=False,preindent=3,CSE_varprefix=tmp_xx")

    # Define the input variables:
    gammaSphorCartDD = ixp.declarerank2("gammaSphorCartDD", "sym01")
    KSphorCartDD     = ixp.declarerank2("KSphorCartDD", "sym01")
    betaSphorCartU = ixp.declarerank1("betaSphorCartU")
    BSphorCartU    = ixp.declarerank1("BSphorCartU")

    # Compute Jacobian to convert to Cartesian coordinates
    Jac_dUCart_dDrfmUD, Jac_dUrfm_dDCartUD = rfm.compute_Jacobian_and_inverseJacobian_tofrom_Cartesian()

    gammaCartDD = rfm.basis_transform_tensorDD_from_rfmbasis_to_Cartesian(Jac_dUrfm_dDCartUD, gammaSphorCartDD)
    KCartDD = rfm.basis_transform_tensorDD_from_rfmbasis_to_Cartesian(Jac_dUrfm_dDCartUD, KSphorCartDD)
    betaCartU = rfm.basis_transform_vectorU_from_rfmbasis_to_Cartesian(Jac_dUCart_dDrfmUD, betaSphorCartU)
    BCartU = rfm.basis_transform_vectorU_from_rfmbasis_to_Cartesian(Jac_dUCart_dDrfmUD, BSphorCartU)

    list_of_output_exprs    = []
    list_of_output_varnames = []
    for i in range(3):
        list_of_output_exprs += [betaCartU[i]]
        list_of_output_varnames += ["betaCartU" + str(i)]
        list_of_output_exprs += [BCartU[i]]
        list_of_output_varnames += ["BCartU" + str(i)]
        for j in range(i, 3):
            list_of_output_exprs += [gammaCartDD[i][j]]
            list_of_output_varnames += ["gammaCartDD" + str(i) + str(j)]
            list_of_output_exprs += [KCartDD[i][j]]
            list_of_output_varnames += ["KCartDD" + str(i) + str(j)]

    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = \
        (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))

    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=3")
    body += r"""
    }
"""
    # Set reference_metric to Cartesian
    par.set_parval_from_str("reference_metric::CoordSystem", "Cartesian")
    rfm.reference_metric()

    # Reset these variables, as they have been defined above in the C code.
    gammaCartDD = ixp.declarerank2("gammaCartDD", "sym01")
    KCartDD     = ixp.declarerank2("KCartDD", "sym01")

    import BSSN.BSSN_in_terms_of_ADM as BitoA
    BitoA.trK_AbarDD_aDD(gammaCartDD, KCartDD)
    BitoA.gammabarDD_hDD(gammaCartDD)
    BitoA.cf_from_gammaDD(gammaCartDD)

    body += r"""
    // Next convert ADM quantities gammaDD & KDD
    //   into BSSN gammabarDD, AbarDD, cf, and trK, in the Cartesian basis.
    REAL gammabarCartDD00,gammabarCartDD01,gammabarCartDD02,gammabarCartDD11,gammabarCartDD12,gammabarCartDD22;
    REAL AbarCartDD00,AbarCartDD01,AbarCartDD02,AbarCartDD11,AbarCartDD12,AbarCartDD22;
    REAL cf, trK;
    {
"""
    list_of_output_exprs    = [BitoA.cf, BitoA.trK]
    list_of_output_varnames = ["cf", "trK"]
    for i in range(3):
        for j in range(i, 3):
            list_of_output_exprs += [BitoA.gammabarDD[i][j]]
            list_of_output_varnames += ["gammabarCartDD" + str(i) + str(j)]
            list_of_output_exprs += [BitoA.AbarDD[i][j]]
            list_of_output_varnames += ["AbarCartDD" + str(i) + str(j)]
    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = \
        (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))
    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=3")
    body += r"""
    }

    const int idx3 = IDX3S(i0,i1,i2);

    // First set the BSSN scalars, as these don't need a basis transform:
    griddata->gridfuncs.y_n_gfs[IDX4ptS(ALPHAGF, idx3)] = alpha;
    griddata->gridfuncs.y_n_gfs[IDX4ptS(TRKGF, idx3)] = trK;
    griddata->gridfuncs.y_n_gfs[IDX4ptS(CFGF, idx3)] = cf;

    // Then set the BSSN vectors/tensors, which require we perform basis transform & rescaling:
    initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale
      (&griddata->params, griddata->xx[0][i0],griddata->xx[1][i1],griddata->xx[2][i2],
       betaCartU0,betaCartU1,betaCartU2, BCartU0,BCartU1,BCartU2,
       gammabarCartDD00,gammabarCartDD01,gammabarCartDD02,gammabarCartDD11,gammabarCartDD12,gammabarCartDD22,
       AbarCartDD00,AbarCartDD01,AbarCartDD02,AbarCartDD11,AbarCartDD12,AbarCartDD22,
       idx3, griddata->gridfuncs.y_n_gfs);
  } // END LOOP over all gridpoints on given grid

  initial_data_lambdaU_grid_interior(&griddata->params, griddata->xx,
                                     griddata->gridfuncs.y_n_gfs);
"""

    # Restore reference_metric to output_Coord
    par.set_parval_from_str("reference_metric::CoordSystem", output_Coord)
    rfm.reference_metric()

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)
    return pickle_NRPy_env()

<a id='basis_xform'></a>

# Step 3: `initial_data_BSSN_basis_transform_Cartesian_to_rfm()`: Convert BSSN vectors/tensors from Cartesian to reference-metric basis \[Back to [top](#toc)\]
$$\label{basis_xform}$$

By the time this function is called, all BSSN tensors and vectors are in the Cartesian coordinate basis $x^i_{\rm Cart} = (x,y,z)$, but we need them in the curvilinear coordinate basis $x^i_{\rm rfm}=$`(xx0,xx1,xx2)` set by the `"reference_metric::CoordSystem"` variable. Empirically speaking, it is far easier to write `(x(xx0,xx1,xx2),y(xx0,xx1,xx2),z(xx0,xx1,xx2))` than the inverse, so we will compute the Jacobian matrix

$$
{\rm Jac\_dUCart\_dDrfmUD[i][j]} = \frac{\partial x^i_{\rm Cart}}{\partial x^j_{\rm rfm}},
$$

via exact differentiation (courtesy SymPy), and the inverse Jacobian
$$
{\rm Jac\_dUrfm\_dDCartUD[i][j]} = \frac{\partial x^i_{\rm rfm}}{\partial x^j_{\rm Cart}},
$$

using NRPy+'s `generic_matrix_inverter3x3()` function. In terms of these, the transformation of BSSN tensors from Spherical to `"reference_metric::CoordSystem"` coordinates may be written:

\begin{align}
\beta^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} \beta^\ell_{\rm Cart}\\
B^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} B^\ell_{\rm Cart}\\
\bar{\gamma}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{\gamma}^{\rm Cart}_{\ell m}\\
\bar{A}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{A}^{\rm Cart}_{\ell m}
\end{align}

The above basis transforms are included in functions `basis_transform_tensorDD_from_Cartesian_to_rfmbasis()` and `basis_transform_vectorU_from_Cartesian_to_rfmbasis()` in `reference_metric.py`, and we use them below.

After the basis transform has been performed, we perform tensor rescalings to compute the evolved variables $h_{ij}$, $a_{ij}$, $\text{vet}^i$, and $\text{bet}^i$:

$\bar{\gamma}_{ij}$ is rescaled $h_{ij}$ according to the prescription described in the [the covariant BSSN formulation tutorial](Tutorial-BSSN_formulation.ipynb) (also [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):

$$
h_{ij} = (\bar{\gamma}_{ij} - \hat{\gamma}_{ij})/\text{ReDD[i][j]}.
$$

Further $\bar{A}_{ij}$, $\beta^i$, $B^i$ are rescaled to $a_{ij}$, $\text{vet}^i$, and $\text{bet}^i$ respectively via the standard formulas (found in [the covariant BSSN formulation tutorial](Tutorial-BSSN_formulation.ipynb); also [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):

\begin{align}
a_{ij} &= \bar{A}_{ij}/\text{ReDD[i][j]} \\
\text{vet}^i &= \beta^i/\text{ReU[i]} \\
\text{bet}^i &= B^i/\text{ReU[i]}.
\end{align}

In [3]:
# By the time this function is called, all BSSN tensors and vectors are in the Cartesian
# coordinate basis $x^i_{\rm Cart} = (x,y,z)$, but we need them in the curvilinear
# coordinate basis $x^i_{\rm rfm}=$`(xx0,xx1,xx2)` set by the
#  `"reference_metric::CoordSystem"` variable.
def add_to_Cfunction_dict_initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale():
    includes = ["NRPy_basic_defines.h"]
    c_type = "void"

    output_Coord = par.parval_from_str("reference_metric::CoordSystem")
    desc = "Basis transform AbarDD and gammabarDD from Cartesian to " + output_Coord + " coordinates"
    name = "initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale"
    params = """const paramstruct *restrict params, const REAL xx0,const REAL xx1,const REAL xx2,
                                                                    const REAL betaCartU0,const REAL betaCartU1,const REAL betaCartU2,
                                                                    const REAL BCartU0,const REAL BCartU1,const REAL BCartU2,
                                                                    const REAL gammabarCartDD00,const REAL gammabarCartDD01,const REAL gammabarCartDD02,
                                                                    const REAL gammabarCartDD11,const REAL gammabarCartDD12,const REAL gammabarCartDD22,
                                                                    const REAL AbarCartDD00,const REAL AbarCartDD01,const REAL AbarCartDD02,
                                                                    const REAL AbarCartDD11,const REAL AbarCartDD12,const REAL AbarCartDD22,
                                                                    const int idx3, REAL *restrict y_n_gfs"""

    # Define the input variables:
    gammabarCartDD = ixp.declarerank2("gammabarCartDD", "sym01")
    AbarCartDD     = ixp.declarerank2("AbarCartDD", "sym01")
    betaCartU = ixp.declarerank1("betaCartU")
    BCartU    = ixp.declarerank1("BCartU")

    # Set reference_metric to the output_Coord
    par.set_parval_from_str("reference_metric::CoordSystem", output_Coord)
    rfm.reference_metric()
    # Compute Jacobian to convert to Cartesian coordinates
    Jac_dUCart_dDrfmUD, Jac_dUrfm_dDCartUD = rfm.compute_Jacobian_and_inverseJacobian_tofrom_Cartesian()

    gammabarDD = rfm.basis_transform_tensorDD_from_Cartesian_to_rfmbasis(Jac_dUCart_dDrfmUD, gammabarCartDD)
    AbarDD = rfm.basis_transform_tensorDD_from_Cartesian_to_rfmbasis(Jac_dUCart_dDrfmUD, AbarCartDD)
    betaU = rfm.basis_transform_vectorU_from_Cartesian_to_rfmbasis(Jac_dUrfm_dDCartUD, betaCartU)
    BU = rfm.basis_transform_vectorU_from_Cartesian_to_rfmbasis(Jac_dUrfm_dDCartUD, BCartU)

    # Next rescale:
    vetU = ixp.zerorank1()
    betU = ixp.zerorank1()
    hDD  = ixp.zerorank2()
    aDD  = ixp.zerorank2()
    for i in range(3):
        vetU[i] = betaU[i] / rfm.ReU[i]
        betU[i] =    BU[i] / rfm.ReU[i]
        for j in range(3):
            hDD[i][j] = (gammabarDD[i][j] - rfm.ghatDD[i][j]) / rfm.ReDD[i][j]
            aDD[i][j] = AbarDD[i][j] / rfm.ReDD[i][j]

    def gfaccess(gfname):
        return "y_n_gfs[IDX4ptS("+gfname.upper()+"GF, idx3)]"
    list_of_output_exprs    = []
    list_of_output_varnames = []
    for i in range(3):
        list_of_output_exprs += [vetU[i]]
        list_of_output_varnames += [gfaccess("vetU" + str(i))]
        list_of_output_exprs += [betU[i]]
        list_of_output_varnames += [gfaccess("betU" + str(i))]
        for j in range(i, 3):
            list_of_output_exprs += [hDD[i][j]]
            list_of_output_varnames += [gfaccess("hDD" + str(i) + str(j))]
            list_of_output_exprs += [aDD[i][j]]
            list_of_output_varnames += [gfaccess("aDD" + str(i) + str(j))]
    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = \
        (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))

    body = outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=1")

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=True)
    return pickle_NRPy_env()

<a id='lambda'></a>

# Step 4: `initial_data_lambdaU_grid_interior()`: Compute $\lambda^i$ from finite-difference derivatives of rescaled metric quantities \[Back to [top](#toc)\]
$$\label{lambda}$$

We compute $\bar{\Lambda}^i$ (Eqs. 4 and 5 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf)), from finite-difference derivatives of rescaled metric quantities $h_{ij}$:

$$
\bar{\Lambda}^i = \bar{\gamma}^{jk}\left(\bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}\right).
$$

The [reference_metric.py](../edit/reference_metric.py) module provides us with analytic expressions for $\hat{\Gamma}^i_{jk}$, so here we need only compute finite-difference expressions for $\bar{\Gamma}^i_{jk}$, based on the values for $h_{ij}$ provided in the initial data. Once $\bar{\Lambda}^i$ has been computed, we apply the usual rescaling procedure:
$$
\lambda^i = \bar{\Lambda}^i/\text{ReU[i]},
$$
and then output the result to a C file using the NRPy+ finite-difference C output routine.

In [4]:
# initial_data_lambdaU_grid_interior() computes lambdaU from
#  finite-difference derivatives of rescaled metric quantities
def add_to_Cfunction_dict_initial_data_lambdaU_grid_interior():
    includes = ["NRPy_basic_defines.h"]
    c_type = "void"

    output_Coord = par.parval_from_str("reference_metric::CoordSystem")
    desc = "Compute lambdaU in " + output_Coord + " coordinates"
    name = "initial_data_lambdaU_grid_interior"
    params = """const paramstruct *restrict params, REAL *restrict xx[3], REAL *restrict in_gfs"""
    # Step 7: Compute $\bar{\Lambda}^i$ from finite-difference derivatives of rescaled metric quantities

    # We will need all BSSN gridfunctions to be defined, as well as
    #     expressions for gammabarDD_dD in terms of exact derivatives of
    #     the rescaling matrix and finite-difference derivatives of
    #     hDD's. This functionality is provided by BSSN.BSSN_unrescaled_and_barred_vars,
    #     which we call here to overwrite above definitions of gammabarDD,gammabarUU, etc.
    Bq.gammabar__inverse_and_derivs()  # Provides gammabarUU and GammabarUDD
    gammabarUU    = Bq.gammabarUU
    GammabarUDD   = Bq.GammabarUDD

    # Next evaluate \bar{\Lambda}^i, based on GammabarUDD above and GammahatUDD
    #       (from the reference metric):
    LambdabarU = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                LambdabarU[i] += gammabarUU[j][k] * (GammabarUDD[i][j][k] - rfm.GammahatUDD[i][j][k])

    # Finally apply rescaling:
    # lambda^i = Lambdabar^i/\text{ReU[i]}
    lambdaU = ixp.zerorank1()
    for i in range(3):
        lambdaU[i] = LambdabarU[i] / rfm.ReU[i]

    lambdaU_expressions = [lhrh(lhs=gri.gfaccess("in_gfs", "lambdaU0"), rhs=lambdaU[0]),
                           lhrh(lhs=gri.gfaccess("in_gfs", "lambdaU1"), rhs=lambdaU[1]),
                           lhrh(lhs=gri.gfaccess("in_gfs", "lambdaU2"), rhs=lambdaU[2])]
    body = fin.FD_outputC("returnstring", lambdaU_expressions,
                           params="outCverbose=False,includebraces=False,preindent=0")
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        loopopts="InteriorPoints,Read_xxs",
        enableCparameters=True)
    return pickle_NRPy_env()

In [5]:
def add_to_Cfunction_dict_exact_ADM_ID_function(IDtype, IDCoordSystem, alpha, betaU, BU, gammaDD, KDD):
    includes = ["NRPy_basic_defines.h"]
    desc = IDtype + " initial data"
    c_type = "void"
    name = IDtype
    params = "const paramstruct *params, const REAL xCart[3], const ID_persist_struct *restrict ID_persist, ID_output_struct *restrict ID_output"
    orig_Coord = par.parval_from_str("reference_metric::CoordSystem")
    par.set_parval_from_str("reference_metric::CoordSystem", IDCoordSystem)
    rfm.reference_metric()
    body = ""
    if IDCoordSystem == "Spherical":
        body += r"""  const REAL Cartx=xCart[0], Carty=xCart[1], Cartz=xCart[2];
  REAL xx0,xx1,xx2 __attribute__((unused));  // xx2 might be unused in the case of axisymmetric initial data.
  {
""" + outputC(rfm.Cart_to_xx[:3], ["xx0", "xx1", "xx2"], filename="returnstring",
              params="outCverbose=False,includebraces=False,preindent=2") + """
  }
  const REAL r  = xx0; // Some ID only specify r,th,ph.
  const REAL th = xx1;
  const REAL ph = xx2;
"""
    elif IDCoordSystem == "Cartesian":
        body += r"""  const REAL xx0=xCart[0], xx1=xCart[1], xx2=xCart[2];
"""
    else:
        print("add_to_Cfunction_dict_exact_ADM_ID_function() Error: IDCoordSystem == " + IDCoordSystem + " unsupported")
        sys.exit(1)
    list_of_output_exprs = [alpha]
    list_of_output_varnames = ["ID_output->alpha"]
    for i in range(3):
        list_of_output_exprs += [betaU[i]]
        list_of_output_varnames += ["ID_output->betaSphorCartU" + str(i)]
        list_of_output_exprs += [BU[i]]
        list_of_output_varnames += ["ID_output->BSphorCartU" + str(i)]
        for j in range(i, 3):
            list_of_output_exprs += [gammaDD[i][j]]
            list_of_output_varnames += ["ID_output->gammaSphorCartDD" + str(i) + str(j)]
            list_of_output_exprs += [KDD[i][j]]
            list_of_output_varnames += ["ID_output->KSphorCartDD" + str(i) + str(j)]
    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = \
        (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))

    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=1")

    # Restore CoordSystem:
    par.set_parval_from_str("reference_metric::CoordSystem", orig_Coord)
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc, c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=True)
    return pickle_NRPy_env()

<a id='nbd'></a>

# Step 5: `register_NRPy_basic_defines()`: Register `ID_data_struct` with `NRPy_basic_defines.h` \[Back to [top](#toc)\]
$$\label{nbd}$$

Other than its core use as a means to store ADM input quantities, `ID_data_struct` is designed to be extensible. For example, it may be used to store e.g., pseudospectral coefficients for TwoPunctures, initial data gridfunctions from NRPyElliptic, pointers to TOV 1D data from the TOV solver, etc.

In [6]:
# Other than its core use as a means to store ADM input quantities,
# `ID_output_struct` is designed to be extensible. For example, it may be
# used to store e.g., pseudospectral coefficients for TwoPunctures,
# initial data gridfunctions from NRPyElliptic, pointers to TOV 1D data
# from the TOV solver, etc.
def register_NRPy_basic_defines(ID_persist_struct_contents_str=""):
    Nbd = r"""typedef struct __ID_output_struct__ {
  REAL alpha;

  REAL betaSphorCartU0, betaSphorCartU1, betaSphorCartU2;
  REAL BSphorCartU0, BSphorCartU1, BSphorCartU2;

  REAL gammaSphorCartDD00, gammaSphorCartDD01, gammaSphorCartDD02;
  REAL gammaSphorCartDD11, gammaSphorCartDD12, gammaSphorCartDD22;

  REAL KSphorCartDD00, KSphorCartDD01, KSphorCartDD02;
  REAL KSphorCartDD11, KSphorCartDD12, KSphorCartDD22;
} ID_output_struct;
"""
    if ID_persist_struct_contents_str == "":
        Nbd += "typedef struct __ID_persist_struct__ {\n"
        Nbd += "} ID_persist_struct;\n"
    else:
        Nbd += ID_persist_struct_contents_str + "\n"
    outC_NRPy_basic_defines_h_dict["BSSN_initial_data"] = Nbd

<a id='code_validation'></a>

# Step 6: Code Validation against  `BSSN.ADM_Initial_Data_Reader__BSSN_Converter` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the C codes for converting "numerical" UIUCBlackHole initial data (in Spherical coordinates/basis) to BSSN Curvilinear data in Cylindrical coordinates/basis between
1. this tutorial and 
2. the NRPy+ [BSSN.ADM_Initial_Data_Reader__BSSN_Converter](../edit/BSSN/ADM_Initial_Data_Reader__BSSN_Converter.py) module.

By default, we analyze these expressions in Cylindrical coordinates, though other coordinate systems may be chosen.

In [7]:
import BSSN.ADM_Initial_Data_Reader__BSSN_Converter as AID

funclist = [(add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart, AID.add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart),
            (add_to_Cfunction_dict_initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale, AID.add_to_Cfunction_dict_initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale),
            (add_to_Cfunction_dict_initial_data_lambdaU_grid_interior, AID.add_to_Cfunction_dict_initial_data_lambdaU_grid_interior),
            (add_to_Cfunction_dict_exact_ADM_ID_function, AID.add_to_Cfunction_dict_exact_ADM_ID_function),
            (register_NRPy_basic_defines, AID.register_NRPy_basic_defines)
           ]

import inspect
for func in funclist:
    # https://stackoverflow.com/questions/20059011/check-if-two-python-functions-are-equal
    if inspect.getsource(func[0]) != inspect.getsource(func[1]):
        with open(func[0].__name__ + "_Jupyter_notebook_version.c", "w") as file:
            file.write(inspect.getsource(func[0]))
        with open(func[1].__name__ + "_Python_module_version.c", "w") as file:
            file.write(inspect.getsource(func[1]))
        print("ERROR: function " + func[0].__name__ + " is not the same as the Ccodegen_library version!")
        print(" For more info, try this:")
        print("diff " + func[0].__name__ + "_Jupyter_notebook_version.c" + " " + func[1].__name__ + "_Python_module_version.c")
        sys.exit(1)

print("PASS! ALL FUNCTIONS ARE IDENTICAL")

PASS! ALL FUNCTIONS ARE IDENTICAL


In [8]:
add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart()
add_to_Cfunction_dict_initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale()
add_to_Cfunction_dict_initial_data_lambdaU_grid_interior()
register_NRPy_basic_defines()

import BSSN.UIUCBlackHole as UIB
UIB.UIUCBlackHole(ComputeADMGlobalsOnly=True,include_NRPy_basic_defines_and_pickle=False)
add_to_Cfunction_dict_exact_ADM_ID_function("UIUCBlackHole", "Spherical",
                                            UIB.alphaSph, UIB.betaSphU, UIB.BSphU, UIB.gammaSphDD, UIB.KSphDD)


import outputC as outC
import finite_difference as fin
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=True, enable_SIMD=False)

outC.outC_NRPy_basic_defines_h_dict["MoL"] = """
typedef struct __MoL_gridfunctions_struct__ {
  REAL *restrict y_n_gfs;
} MoL_gridfunctions_struct;
"""
par.register_NRPy_basic_defines()  # add `paramstruct params` to griddata struct.
list_of_extras_in_griddata_struct = ["MoL_gridfunctions_struct gridfuncs;"]
gri.register_C_functions_and_NRPy_basic_defines(list_of_extras_in_griddata_struct=list_of_extras_in_griddata_struct)  # #define IDX3S(), etc.

outC.outputC_register_C_functions_and_NRPy_basic_defines()
outC.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(os.path.join(Ccodesrootdir))

# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(Ccodesrootdir, enable_SIMD=False)
outC.construct_NRPy_function_prototypes_h(Ccodesrootdir)

outC.construct_Makefile_from_outC_function_dict(Ccodesrootdir, "", uses_free_parameters_h=False,
                                                compiler_opt_option="fastdebug", addl_CFLAGS=None,
                                                mkdir_Ccodesrootdir=True, use_make=True, CC="gcc",
                                                create_lib=True,  include_dirs=None)
# Now all the C codes generated in this notebook may be found in ADM_initial_data_to_BSSN_converter/
#    Go there and type `make` to see if it all compiles.

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

Once the following code finishes running, the generated PDF may be found at the following location within the directory you have the NRPy+ tutorial saved:
[Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.pdf](Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.pdf)

In [9]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-ADM_Initial_Data_Reader__BSSN_Converter")

Created Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.tex, and compiled
    LaTeX file to PDF file Tutorial-
    ADM_Initial_Data_Reader__BSSN_Converter.pdf
